In [ ]:
%load_ext rich
%load_ext autoreload
%autoreload 2

In [ ]:
import polars as pl
import pointblank as pb

from odyssey.core import *
from odyssey.explore import *

In [ ]:
from config.paths import INTERIM_DATA

In [ ]:
g220 = Dataset("G220_Q.sav", INTERIM_DATA)
lf_g220, _meta = g220.load_data()
df_g220 = lf_g220.collect()

In [ ]:
g222 = Dataset("G222_Q.sav", INTERIM_DATA)
lf_g222, _meta = g222.load_data()
df_g222 = lf_g222.collect()

In [ ]:
from typing import Callable

def check_total_mins(
    hpd_column: str,
    mpd_column: str
    ) -> Callable:
    """
    Returns a preprocessing function to verify the minutes for a given category have been correctly calculated.
    Cap the total at 180 minutes, and preserve null values.
    """
    def preprocessor(df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            (pl.col(hpd_column).fill_null(0) * 60 + pl.col(mpd_column).fill_null(0))
            .pipe(lambda expr: pl.when(expr > 180).then(180).otherwise(expr))
            .alias("check")
        )
    return preprocessor

def check_met(
    mins_column: str, 
    n_days_column: str, 
    met_column: str,
    factor: int|float # the corresponding factor for the activity (Vig: 8, Mod: 4, Walk: 3.3)
    ) -> Callable:
    """Returns a preprocessing function to verify the calculated MET value for a given category."""
    def preprocessor(df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            (pl.col(mins_column).fill_null(0) * pl.col(n_days_column).fill_null(0) * factor).alias("check"),
            pl.col(met_column).fill_null(0)
        )
    return preprocessor

def check_tot_met(
    met_columns: list[str],
    tot_met_column: str
    ) -> Callable:
    """Returns a preprocessing function to verify the calculated total MET value."""
    def preprocessor(df: pl.DataFrame) -> pl.DataFrame:
        expr = sum(pl.col(col).fill_null(0) for col in met_columns)
        
        return df.with_columns(
            expr.alias("check"),
            pl.col(tot_met_column).fill_null(0) # Fill nulls with 0; otherwise the validation skips if one value in a comparison is null
        )
    return preprocessor

In [ ]:
def validate_ipaq(
    prefix: str, # prefix for the dataset
    df: pl.DataFrame
    ) -> pb.Validate:

    validation = (
        pb.Validate(
            data=df,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_VIG_MINS",
            value=pb.col("check"),
            pre=check_total_mins(f"{prefix}_IPAQ_VIG_HPD", f"{prefix}_IPAQ_VIG_MPD"),
            brief="Check total mins/day equals `HPD*60 + MPD`",
            na_pass=True,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_MOD_MINS",
            value=pb.col("check"),
            pre=check_total_mins(f"{prefix}_IPAQ_MOD_HPD", f"{prefix}_IPAQ_MOD_MPD"),
            brief="Check total mins/day equals `HPD*60 + MPD`",
            na_pass=True,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_WALK_MINS",
            value=pb.col("check"),
            pre=check_total_mins(f"{prefix}_IPAQ_WALK_HPD", f"{prefix}_IPAQ_WALK_MPD"),
            brief="Check total mins/day equals `HPD*60 + MPD`",
            na_pass=True,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_VIG_MET",
            value=pb.col("check"),
            pre=check_met(f"{prefix}_IPAQ_VIG_MINS", f"{prefix}_IPAQ_VIG_D", f"{prefix}_IPAQ_VIG_MET", factor=8),
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_MOD_MET",
            value=pb.col("check"),
            pre=check_met(f"{prefix}_IPAQ_MOD_MINS", f"{prefix}_IPAQ_MOD_D", f"{prefix}_IPAQ_MOD_MET", factor=4),
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_WALK_MET",
            value=pb.col("check"),
            pre=check_met(f"{prefix}_IPAQ_WALK_MINS", f"{prefix}_IPAQ_WALK_D", f"{prefix}_IPAQ_WALK_MET", factor=3.3),
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_TOT_MET",
            value=pb.col("check"),
            pre=check_tot_met([f"{prefix}_IPAQ_VIG_MET", f"{prefix}_IPAQ_MOD_MET", f"{prefix}_IPAQ_WALK_MET"], f"{prefix}_IPAQ_TOT_MET"),
            brief="Check `TOT_MET` equals the sum of `VIG_MET`, `MOD_MET`, and `WALK_MET`"
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_VIG_D",
            left=1,
            right=7,
            segments=(f"{prefix}_IPAQ_VIG_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_VIG_HPD", 
            left=0,
            right=18, # unrealistic to do more than 18 hours of exercise per day (even that is a stretch!!)
            segments=(f"{prefix}_IPAQ_VIG_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_VIG_MPD", 
            left=0,
            right=59,
            segments=(f"{prefix}_IPAQ_VIG_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_VIG_MINS", 
            left=0,
            right=180, # total mins per category is capped at 180 mins
            segments=(f"{prefix}_IPAQ_VIG_W", 1),
        )
        .col_vals_eq(
            columns=[f"{prefix}_IPAQ_VIG_D", f"{prefix}_IPAQ_VIG_HPD", f"{prefix}_IPAQ_VIG_MPD", f"{prefix}_IPAQ_VIG_MINS", f"{prefix}_IPAQ_VIG_MET"], 
            value=0,
            segments=(f"{prefix}_IPAQ_VIG_W", 0)
        )
        .col_vals_null(
            columns=[f"{prefix}_IPAQ_VIG_D", f"{prefix}_IPAQ_VIG_HPD", f"{prefix}_IPAQ_VIG_MPD", f"{prefix}_IPAQ_VIG_MINS", f"{prefix}_IPAQ_VIG_MET"],
            pre=lambda df: df.with_columns(pl.col(f"{prefix}_IPAQ_VIG_W").fill_null(-1)), # Pointblank doesn't seem to like segmenting values with null, so transform null to -1 and segment y that
            segments=(f"{prefix}_IPAQ_VIG_W", -1)
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_MOD_D",
            left=1,
            right=7,
            segments=(f"{prefix}_IPAQ_MOD_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_MOD_HPD", 
            left=0,
            right=18, # unrealistic to do more than 18 hours of exercise per day (even that is a stretch!!)
            segments=(f"{prefix}_IPAQ_MOD_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_MOD_MPD", 
            left=0,
            right=59,
            segments=(f"{prefix}_IPAQ_MOD_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_MOD_MINS", 
            left=0,
            right=180, # total mins per category is capped at 180 mins
            segments=(f"{prefix}_IPAQ_MOD_W", 1),
        )
        .col_vals_eq(
            columns=[f"{prefix}_IPAQ_MOD_D", f"{prefix}_IPAQ_MOD_HPD", f"{prefix}_IPAQ_MOD_MPD", f"{prefix}_IPAQ_MOD_MINS", f"{prefix}_IPAQ_MOD_MET"], 
            value=0,
            segments=(f"{prefix}_IPAQ_MOD_W", 0)
        )
        .col_vals_null(
            columns=[f"{prefix}_IPAQ_MOD_D", f"{prefix}_IPAQ_MOD_HPD", f"{prefix}_IPAQ_MOD_MPD", f"{prefix}_IPAQ_MOD_MINS", f"{prefix}_IPAQ_MOD_MET"],
            pre=lambda df: df.with_columns(pl.col(f"{prefix}_IPAQ_MOD_W").fill_null(-1)), # Pointblank doesn't seem to like segmenting values with null, so transform null to -1 and segment y that
            segments=(f"{prefix}_IPAQ_MOD_W", -1)
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_WALK_D",
            left=1,
            right=7,
            segments=(f"{prefix}_IPAQ_WALK_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_WALK_HPD", 
            left=0,
            right=18, # unrealistic to do more than 18 hours of exercise per day (even that is a stretch!!)
            segments=(f"{prefix}_IPAQ_WALK_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_WALK_MPD", 
            left=0,
            right=59,
            segments=(f"{prefix}_IPAQ_WALK_W", 1),
        )
        .col_vals_between(
            columns=f"{prefix}_IPAQ_WALK_MINS", 
            left=0,
            right=180, # total mins per category is capped at 180 mins
            segments=(f"{prefix}_IPAQ_WALK_W", 1),
        )
        .col_vals_eq(
            columns=[f"{prefix}_IPAQ_WALK_D", f"{prefix}_IPAQ_WALK_HPD", f"{prefix}_IPAQ_WALK_MPD", f"{prefix}_IPAQ_WALK_MINS", f"{prefix}_IPAQ_WALK_MET"], 
            value=0,
            segments=(f"{prefix}_IPAQ_WALK_W", 0)
        )
        .col_vals_null(
            columns=[f"{prefix}_IPAQ_WALK_D", f"{prefix}_IPAQ_WALK_HPD", f"{prefix}_IPAQ_WALK_MPD", f"{prefix}_IPAQ_WALK_MINS", f"{prefix}_IPAQ_WALK_MET"],
            pre=lambda df: df.with_columns(pl.col(f"{prefix}_IPAQ_WALK_W").fill_null(-1)), # Pointblank doesn't seem to like segmenting values with null, so transform null to -1 and segment y that
            segments=(f"{prefix}_IPAQ_WALK_W", -1)
        )
    )

    return validation

In [ ]:
validation = validate_ipaq(prefix="G222", df=df_g222).interrogate()